# Multi-step Prediction

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import pickle

from ta.momentum import rsi, stoch, stoch_signal
from ta.volatility import average_true_range
from ta.volume import on_balance_volume

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from keras.layers import MaxPooling1D, Flatten, Conv1D
from keras.wrappers.scikit_learn import KerasRegressor

import scipy.stats as stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from scipy.stats import normaltest
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pmdarima import auto_arima

2024-10-03 09:56:04.583340: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Data preparation

In [2]:
df_nasdaq = pd.read_csv('df_nasdaq.csv')
df_sp500 = pd.read_csv('df_sp500.csv')
df_dji = pd.read_csv('df_dji.csv')

stocks = {'df_nasdaq': df_nasdaq, 'df_sp500': df_sp500, 'df_dji': df_dji}

for name, df in stocks.items():
    print(f'Shape of {name}: {df.shape}')

Shape of df_nasdaq: (1372, 19)
Shape of df_sp500: (1372, 19)
Shape of df_dji: (1372, 19)


In [3]:
feature = ['Open', 'High', 'Low', 'Volume', 'DailyReturn', 'Adj Close',
           'MA_10', 'MA_20', 'MA_50', 'EMA_100', 'EMA_200', 
           'Volatility_bbh', 'Volatility_bbl', 'MACD', 'RSI', 'ATR','OBV']
target = 'Close'

# Result of XGBoost feature selection, according to 7600_code_XGB_part
feature_xgb_nasdaq = ['Adj Close', 'High', 'Low', 'Open', 'MA_10', 'Volatility_bbl']
feature_xgb_sp500 = ['Adj Close', 'High', 'Low', 'Open', 'MA_10', 'MA_20']
feature_xgb_dji = ['Adj Close', 'High', 'Low', 'Open', 'MA_10', 'OBV']

df_nasdaq_preprocessed = df_nasdaq[[target] + feature_xgb_nasdaq]
df_sp500_preprocessed = df_sp500[[target] + feature_xgb_sp500]
df_dji_preprocessed = df_dji[[target] + feature_xgb_dji]

In [4]:
def split_data(stock):
    stock = stock.to_numpy()
    test_set_size = int(np.round(0.2 * len(stock)))
    val_set_size = int(np.round(0.2 * len(stock)))
    train_set_size = len(stock) - test_set_size - val_set_size
    
    train = stock[:train_set_size, :]
    val = stock[train_set_size: train_set_size+val_set_size, :]
    test = stock[train_set_size+val_set_size:, :]

    return (train, val, test)

def createXY(dataset, lookback, n_step):
    dataX, dataY = [], []
    for i in range(lookback, len(dataset) - n_step + 1):
        dataX.append(dataset[i - lookback:i, 0:dataset.shape[1]])  # Features over the lookback period
        dataY.append(dataset[i:i + n_step, 0])  # Predict next 'n_step' values
    return np.array(dataX), np.array(dataY)

# Define datasets
datasets = {
    'nasdaq': df_nasdaq_preprocessed,
    'sp500': df_sp500_preprocessed,
    'dji': df_dji_preprocessed
}

scaled_data = {}

lookback = 30
n_steps = 30

scaler_nasdaq = MinMaxScaler(feature_range=(0, 1))
scaler_sp500 = MinMaxScaler(feature_range=(0, 1))
scaler_dji = MinMaxScaler(feature_range=(0, 1))

for stock_name, df_stock in datasets.items():
    train, val, test = split_data(df_stock)
    
    if stock_name == 'nasdaq':
        scaled_train = scaler_nasdaq.fit_transform(train)
        scaled_val = scaler_nasdaq.transform(val)
        scaled_test = scaler_nasdaq.transform(test)
    elif stock_name == 'sp500':
        scaled_train = scaler_sp500.fit_transform(train)
        scaled_val = scaler_sp500.transform(val)
        scaled_test = scaler_sp500.transform(test)
    elif stock_name == 'dji':
        scaled_train = scaler_dji.fit_transform(train)
        scaled_val = scaler_dji.transform(val)
        scaled_test = scaler_dji.transform(test)
    
    X_train, y_train = createXY(scaled_train, lookback, n_steps)
    X_val, y_val = createXY(scaled_val, lookback, n_steps)
    X_test, y_test = createXY(scaled_test, lookback, n_steps)
    
    scaled_data[stock_name] = {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test
    }

# Check the results
for stock in scaled_data:
    print(f"Data for {stock}:")
    print(f"X_train shape: {scaled_data[stock]['X_train'].shape}")
    print(f"y_train shape: {scaled_data[stock]['y_train'].shape}")
    print(f"X_val shape: {scaled_data[stock]['X_val'].shape}")
    print(f"y_val shape: {scaled_data[stock]['y_val'].shape}")
    print(f"X_test shape: {scaled_data[stock]['X_test'].shape}")
    print(f"y_test shape: {scaled_data[stock]['y_test'].shape}\n")

Data for nasdaq:
X_train shape: (765, 30, 7)
y_train shape: (765, 30)
X_val shape: (215, 30, 7)
y_val shape: (215, 30)
X_test shape: (215, 30, 7)
y_test shape: (215, 30)

Data for sp500:
X_train shape: (765, 30, 7)
y_train shape: (765, 30)
X_val shape: (215, 30, 7)
y_val shape: (215, 30)
X_test shape: (215, 30, 7)
y_test shape: (215, 30)

Data for dji:
X_train shape: (765, 30, 7)
y_train shape: (765, 30)
X_val shape: (215, 30, 7)
y_val shape: (215, 30)
X_test shape: (215, 30, 7)
y_test shape: (215, 30)



In [5]:
'''
load single-prediction model
model_lstm_nasdaq = load_model('lstm/nasdaq_lstm_epoch_350.h5')
model_lstm_sp500 = load_model('lstm/sp500_lstm_epoch_400.h5')
model_lstm_dji = load_model('lstm/dji_lstm_epoch_450.h5')

model_bi_lstm_nasdaq = load_model('bi-lstm/nasdaq_bi-lstm_epoch_050.h5')
model_bi_lstm_sp500 = load_model('bi-lstm/sp500_bi-lstm_epoch_050.h5')
model_bi_lstm_dji = load_model('bi-lstm/dji_bi-lstm_epoch_050.h5')

model_cnn_lstm_nasdaq = load_model('cnn-lstm/nasdaq_cnn-lstm_epoch_050.h5')
model_cnn_lstm_sp500 = load_model('cnn-lstm/sp500_cnn-lstm_epoch_050.h5')
model_cnn_lstm_dji = load_model('cnn-lstm/dji_cnn-lstm_epoch_050.h5')

model_gru_nasdaq = load_model('gru/nasdaq_gru_epoch_300.h5')
model_gru_sp500 = load_model('gru/sp500_gru_epoch_300.h5')
model_gru_dji = load_model('gru/dji_gru_epoch_450.h5')
'''

"\nload single-prediction model\nmodel_lstm_nasdaq = load_model('lstm/nasdaq_lstm_epoch_350.h5')\nmodel_lstm_sp500 = load_model('lstm/sp500_lstm_epoch_400.h5')\nmodel_lstm_dji = load_model('lstm/dji_lstm_epoch_450.h5')\n\nmodel_bi_lstm_nasdaq = load_model('bi-lstm/nasdaq_bi-lstm_epoch_050.h5')\nmodel_bi_lstm_sp500 = load_model('bi-lstm/sp500_bi-lstm_epoch_050.h5')\nmodel_bi_lstm_dji = load_model('bi-lstm/dji_bi-lstm_epoch_050.h5')\n\nmodel_cnn_lstm_nasdaq = load_model('cnn-lstm/nasdaq_cnn-lstm_epoch_050.h5')\nmodel_cnn_lstm_sp500 = load_model('cnn-lstm/sp500_cnn-lstm_epoch_050.h5')\nmodel_cnn_lstm_dji = load_model('cnn-lstm/dji_cnn-lstm_epoch_050.h5')\n\nmodel_gru_nasdaq = load_model('gru/nasdaq_gru_epoch_300.h5')\nmodel_gru_sp500 = load_model('gru/sp500_gru_epoch_300.h5')\nmodel_gru_dji = load_model('gru/dji_gru_epoch_450.h5')\n"

### Fit and prediction

In [12]:
index_name = 'dji'
X_train = scaled_data[index_name]['X_train']
y_train = scaled_data[index_name]['y_train']
X_test = scaled_data[index_name]['X_test']
y_test = scaled_data[index_name]['y_test']

# Fit lstm model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(n_steps))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse', 'mae'])
model.summary()

history = model.fit(X_train, y_train, epochs=100, batch_size=16)
model.save(f'multi-step prediction/{index_name}_lstm_model.h5')
print(f"Model saved at 'multi-step prediction/{index_name}_lstm_model.h5'")

# Predict
initial_sequence = X_test[:1]  # shape: (1, lookback, features)
predicted_steps = model.predict(initial_sequence)
predicted_steps = predicted_steps.flatten()
predicted_steps_copies_array = np.repeat(predicted_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

y_test_steps = y_test[:n_steps]
y_test_steps_copies_array = np.repeat(y_test_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

if index_name == 'nasdaq':
    predicted_steps_inversed = scaler_nasdaq.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_nasdaq.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'sp500':
    predicted_steps_inversed = scaler_sp500.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_sp500.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'dji':
    predicted_steps_inversed = scaler_dji.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_dji.inverse_transform(y_test_steps_copies_array)[:, 0]

results_df = pd.DataFrame({
    'test': y_test_steps_inversed[:n_steps],  # Only first 30 data points
    'lstm': predicted_steps_inversed[:n_steps]  # Only first 30 predicted values
})

results_df.to_csv(f'multi-step prediction/{index_name}_lstm_multi_step_predictions.csv', index=False)

print(f"Predictions saved to 'multi-step prediction/{index_name}_lstm_multi_step_predictions.csv'")

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 50)            11600     
                                                                 
 dropout_10 (Dropout)        (None, 30, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_11 (Dropout)        (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 30)                1530      
                                                                 
Total params: 33,330
Trainable params: 33,330
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
48/48 [==============================]

In [13]:
index_name = 'dji'
X_train = scaled_data[index_name]['X_train']
y_train = scaled_data[index_name]['y_train']
X_test = scaled_data[index_name]['X_test']
y_test = scaled_data[index_name]['y_test']

# Fit bi-lstm model
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.2))
model.add(Dense(n_steps))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse', 'mae'])
model.summary()

history = model.fit(X_train, y_train, epochs=100, batch_size=16)
model.save(f'multi-step prediction/{index_name}_bi-lstm_model.h5')
print(f"Model saved at 'multi-step prediction/{index_name}_bi-lstm_model.h5'")

# Predict
initial_sequence = X_test[:1]  # shape: (1, lookback, features)
predicted_steps = model.predict(initial_sequence)
predicted_steps = predicted_steps.flatten()
predicted_steps_copies_array = np.repeat(predicted_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

y_test_steps = y_test[:n_steps]
y_test_steps_copies_array = np.repeat(y_test_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

if index_name == 'nasdaq':
    predicted_steps_inversed = scaler_nasdaq.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_nasdaq.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'sp500':
    predicted_steps_inversed = scaler_sp500.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_sp500.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'dji':
    predicted_steps_inversed = scaler_dji.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_dji.inverse_transform(y_test_steps_copies_array)[:, 0]

results_df = pd.DataFrame({
    'test': y_test_steps_inversed[:n_steps],  # Only first 30 data points
    'lstm': predicted_steps_inversed[:n_steps]  # Only first 30 predicted values
})

results_df.to_csv(f'multi-step prediction/{index_name}_bi-lstm_multi_step_predictions.csv', index=False)

print(f"Predictions saved to 'multi-step prediction/{index_name}_bi-lstm_multi_step_predictions.csv'")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 30, 100)          23200     
 nal)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 30, 100)           0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 100)              60400     
 nal)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 30)                3030      
                                                                 
Total params: 86,630
Trainable params: 86,630
Non-trai

In [14]:
index_name = 'dji'
X_train = scaled_data[index_name]['X_train']
y_train = scaled_data[index_name]['y_train']
X_test = scaled_data[index_name]['X_test']
y_test = scaled_data[index_name]['y_test']

# Fit cnn-lstm model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(2))
# Don't need flatten

model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(n_steps))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse', 'mae'])
model.summary()

history = model.fit(X_train, y_train, epochs=100, batch_size=16)
model.save(f'multi-step prediction/{index_name}_cnn-lstm_model.h5')
print(f"Model saved at 'multi-step prediction/{index_name}_cnn-lstm_model.h5'")

# Predict
initial_sequence = X_test[:1]  # shape: (1, lookback, features)
predicted_steps = model.predict(initial_sequence)
predicted_steps = predicted_steps.flatten()
predicted_steps_copies_array = np.repeat(predicted_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

y_test_steps = y_test[:n_steps]
y_test_steps_copies_array = np.repeat(y_test_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

if index_name == 'nasdaq':
    predicted_steps_inversed = scaler_nasdaq.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_nasdaq.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'sp500':
    predicted_steps_inversed = scaler_sp500.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_sp500.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'dji':
    predicted_steps_inversed = scaler_dji.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_dji.inverse_transform(y_test_steps_copies_array)[:, 0]

results_df = pd.DataFrame({
    'test': y_test_steps_inversed[:n_steps],  # Only first 30 data points
    'lstm': predicted_steps_inversed[:n_steps]  # Only first 30 predicted values
})

results_df.to_csv(f'multi-step prediction/{index_name}_cnn-lstm_multi_step_predictions.csv', index=False)

print(f"Predictions saved to 'multi-step prediction/{index_name}_cnn-lstm_multi_step_predictions.csv'")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 28, 32)            704       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 14, 32)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 12, 64)            6208      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 6, 64)            0         
 1D)                                                             
                                                                 
 lstm_10 (LSTM)              (None, 6, 50)             23000     
                                                                 
 dropout_14 (Dropout)        (None, 6, 50)            

In [15]:
index_name = 'dji'
X_train = scaled_data[index_name]['X_train']
y_train = scaled_data[index_name]['y_train']
X_test = scaled_data[index_name]['X_test']
y_test = scaled_data[index_name]['y_test']

# Fit gru model
model = Sequential()
model.add(GRU(units=32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=32))
model.add(Dropout(0.2))
model.add(Dense(n_steps))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse', 'mae'])
model.summary()

history = model.fit(X_train, y_train, epochs=100, batch_size=16)
model.save(f'multi-step prediction/{index_name}_gru_model.h5')
print(f"Model saved at 'multi-step prediction/{index_name}_gru_model.h5'")

# Predict
initial_sequence = X_test[:1]  # shape: (1, lookback, features)
predicted_steps = model.predict(initial_sequence)
predicted_steps = predicted_steps.flatten()
predicted_steps_copies_array = np.repeat(predicted_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

y_test_steps = y_test[:n_steps]
y_test_steps_copies_array = np.repeat(y_test_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

if index_name == 'nasdaq':
    predicted_steps_inversed = scaler_nasdaq.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_nasdaq.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'sp500':
    predicted_steps_inversed = scaler_sp500.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_sp500.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'dji':
    predicted_steps_inversed = scaler_dji.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_dji.inverse_transform(y_test_steps_copies_array)[:, 0]

results_df = pd.DataFrame({
    'test': y_test_steps_inversed[:n_steps],  # Only first 30 data points
    'lstm': predicted_steps_inversed[:n_steps]  # Only first 30 predicted values
})

results_df.to_csv(f'multi-step prediction/{index_name}_gru_multi_step_predictions.csv', index=False)

print(f"Predictions saved to 'multi-step prediction/{index_name}_gru_multi_step_predictions.csv'")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 30, 32)            3936      
                                                                 
 dropout_16 (Dropout)        (None, 30, 32)            0         
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dropout_17 (Dropout)        (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 30)                990       
                                                                 
Total params: 11,262
Trainable params: 11,262
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
48/48 [==============================]

In [16]:
import tensorflow as tf
from tensorflow.keras import layers

class SRUCell(layers.Layer):
    def __init__(self, units, **kwargs):
        super(SRUCell, self).__init__(**kwargs)
        self.units = units
        self.state_size = self.units  # Define the state size

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], self.units * 3),
                                 initializer='glorot_uniform',
                                 name='W')
        self.U = self.add_weight(shape=(self.units, self.units * 2),
                                 initializer='orthogonal',
                                 name='U')
        self.b = self.add_weight(shape=(self.units * 3,),
                                 initializer='zeros',
                                 name='b')
        # New weight for transforming input to match hidden state dimension
        self.V = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='glorot_uniform',
                                 name='V')
        super(SRUCell, self).build(input_shape)

    def call(self, inputs, states):
        h_prev = states[0]
        x_W = tf.matmul(inputs, self.W) + self.b
        z, f, r = tf.split(x_W, 3, axis=-1)
        
        h_U = tf.matmul(h_prev, self.U)
        z_U, f_U = tf.split(h_U, 2, axis=-1)
        
        f = tf.sigmoid(f + f_U)
        r = tf.sigmoid(r)
        z = tf.tanh(z + z_U)
        
        # Transform inputs to match the hidden state dimension
        inputs_transformed = tf.matmul(inputs, self.V) 
        h = f * h_prev + (1 - f) * z
        h = r * h + (1 - r) * inputs_transformed
        
        return h, [h]

    def get_config(self):
        config = super(SRUCell, self).get_config()
        config.update({
            'units': self.units,
        })
        return config

class SRU(layers.RNN):
    def __init__(self, units, **kwargs):
        cell = SRUCell(units)
        super(SRU, self).__init__(cell, **kwargs)

    def get_config(self):
        config = super(SRU, self).get_config()
        config.update({
            'units': self.cell.units,
        })
        return config

In [17]:
index_name = 'dji'
X_train = scaled_data[index_name]['X_train']
y_train = scaled_data[index_name]['y_train']
X_test = scaled_data[index_name]['X_test']
y_test = scaled_data[index_name]['y_test']

# Fit sru model
model = Sequential()
model.add(SRU(units=32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.Dropout(0.2))
model.add(SRU(units=32))
model.add(layers.Dropout(0.2))
model.add(Dense(n_steps))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse', 'mae'])
model.summary()

history = model.fit(X_train, y_train, epochs=100, batch_size=16)
model.save(f'multi-step prediction/{index_name}_sru_model.h5')
print(f"Model saved at 'multi-step prediction/{index_name}_sru_model.h5'")

# Predict
initial_sequence = X_test[:1]  # shape: (1, lookback, features)
predicted_steps = model.predict(initial_sequence)
predicted_steps = predicted_steps.flatten()
predicted_steps_copies_array = np.repeat(predicted_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

y_test_steps = y_test[:n_steps]
y_test_steps_copies_array = np.repeat(y_test_steps.reshape(-1, 1), X_train.shape[2], axis=-1)

if index_name == 'nasdaq':
    predicted_steps_inversed = scaler_nasdaq.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_nasdaq.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'sp500':
    predicted_steps_inversed = scaler_sp500.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_sp500.inverse_transform(y_test_steps_copies_array)[:, 0]
elif index_name == 'dji':
    predicted_steps_inversed = scaler_dji.inverse_transform(predicted_steps_copies_array)[:, 0]
    y_test_steps_inversed = scaler_dji.inverse_transform(y_test_steps_copies_array)[:, 0]

results_df = pd.DataFrame({
    'test': y_test_steps_inversed[:n_steps],  # Only first 30 data points
    'lstm': predicted_steps_inversed[:n_steps]  # Only first 30 predicted values
})

results_df.to_csv(f'multi-step prediction/{index_name}_sru_multi_step_predictions.csv', index=False)

print(f"Predictions saved to 'multi-step prediction/{index_name}_sru_multi_step_predictions.csv'")

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sru_2 (SRU)                 (None, 30, 32)            3040      
                                                                 
 dropout_18 (Dropout)        (None, 30, 32)            0         
                                                                 
 sru_3 (SRU)                 (None, 32)                6240      
                                                                 
 dropout_19 (Dropout)        (None, 32)                0         
                                                                 
 dense_9 (Dense)             (None, 30)                990       
                                                                 
Total params: 10,270
Trainable params: 10,270
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
48/48 [==============================]

## Summary

In [18]:
multi_prediction_arima = pd.read_csv('multi-step prediction/arima_predictions.csv')

multi_prediction_nasdaq_lstm = pd.read_csv('multi-step prediction/nasdaq_lstm_multi_step_predictions.csv')
multi_prediction_nasdaq_bi_lstm = pd.read_csv('multi-step prediction/nasdaq_bi-lstm_multi_step_predictions.csv')
multi_prediction_nasdaq_cnn_lstm = pd.read_csv('multi-step prediction/nasdaq_cnn-lstm_multi_step_predictions.csv')
multi_prediction_nasdaq_gru = pd.read_csv('multi-step prediction/nasdaq_gru_multi_step_predictions.csv')
multi_prediction_nasdaq_sru = pd.read_csv('multi-step prediction/nasdaq_sru_multi_step_predictions.csv')

multi_prediction_sp500_lstm = pd.read_csv('multi-step prediction/sp500_lstm_multi_step_predictions.csv')
multi_prediction_sp500_bi_lstm = pd.read_csv('multi-step prediction/sp500_bi-lstm_multi_step_predictions.csv')
multi_prediction_sp500_cnn_lstm = pd.read_csv('multi-step prediction/sp500_cnn-lstm_multi_step_predictions.csv')
multi_prediction_sp500_gru = pd.read_csv('multi-step prediction/sp500_gru_multi_step_predictions.csv')
multi_prediction_sp500_sru = pd.read_csv('multi-step prediction/sp500_sru_multi_step_predictions.csv')

multi_prediction_dji_lstm = pd.read_csv('multi-step prediction/dji_lstm_multi_step_predictions.csv')
multi_prediction_dji_bi_lstm = pd.read_csv('multi-step prediction/dji_bi-lstm_multi_step_predictions.csv')
multi_prediction_dji_cnn_lstm = pd.read_csv('multi-step prediction/dji_cnn-lstm_multi_step_predictions.csv')
multi_prediction_dji_gru = pd.read_csv('multi-step prediction/dji_gru_multi_step_predictions.csv')
multi_prediction_dji_sru = pd.read_csv('multi-step prediction/dji_sru_multi_step_predictions.csv')

In [22]:
df_nasdaq_multi_prediction = pd.DataFrame({
    'test': multi_prediction_nasdaq_lstm.iloc[:, 0],  
    'lstm': multi_prediction_nasdaq_lstm.iloc[:, 1],  
    'bi_lstm': multi_prediction_nasdaq_bi_lstm.iloc[:, 1],  
    'cnn_lstm': multi_prediction_nasdaq_cnn_lstm.iloc[:, 1],  
    'gru': multi_prediction_nasdaq_gru.iloc[:, 1], 
    'sru': multi_prediction_nasdaq_sru.iloc[:, 1],
    'arima': multi_prediction_arima['nasdaq_pred']
})

df_sp500_multi_prediction = pd.DataFrame({
    'test': multi_prediction_sp500_lstm.iloc[:, 0],  
    'lstm': multi_prediction_sp500_lstm.iloc[:, 1],  
    'bi_lstm': multi_prediction_sp500_bi_lstm.iloc[:, 1],  
    'cnn_lstm': multi_prediction_sp500_cnn_lstm.iloc[:, 1],  
    'gru': multi_prediction_sp500_gru.iloc[:, 1], 
    'sru': multi_prediction_sp500_sru.iloc[:, 1],
    'arima': multi_prediction_arima['sp500_pred']
})

df_dji_multi_prediction = pd.DataFrame({
    'test': multi_prediction_dji_lstm.iloc[:, 0],  
    'lstm': multi_prediction_dji_lstm.iloc[:, 1],  
    'bi_lstm': multi_prediction_dji_bi_lstm.iloc[:, 1],  
    'cnn_lstm': multi_prediction_dji_cnn_lstm.iloc[:, 1],  
    'gru': multi_prediction_dji_gru.iloc[:, 1], 
    'sru': multi_prediction_dji_sru.iloc[:, 1],
    'arima': multi_prediction_arima['dji_pred']
})

In [25]:
model_name = ['lstm', 'bi_lstm', 'cnn_lstm', 'gru', 'sru', 'arima']

mse_nasdaq = []
for model in model_name: 
    mse_nasdaq.append(mean_squared_error(df_nasdaq_multi_prediction['test'], df_nasdaq_multi_prediction[model]))
    
mse_sp500 = []
for model in model_name: 
    mse_sp500.append(mean_squared_error(df_sp500_multi_prediction['test'], df_sp500_multi_prediction[model]))
    
mse_dji = []
for model in model_name: 
    mse_dji.append(mean_squared_error(df_dji_multi_prediction['test'], df_dji_multi_prediction[model]))

df_mse = pd.DataFrame({'nasdaq': mse_nasdaq, 'sp500': mse_sp500, 'dji': mse_dji}, index=model_name).round(2)

In [35]:
df_mse

,nasdaq,sp500,dji
lstm,539478.91,13471.48,405008.05
bi_lstm,1487873.36,13534.70,706007.66
cnn_lstm,632002.78,10870.95,256077.69
gru,435951.36,9537.11,245071.49
sru,359850.86,18552.14,187336.65
arima,351821.42,23711.74,804533.47


In [34]:
min_mse_model = df_mse.idxmin()
min_mse_value = df_mse.min()

for index, model in min_mse_model.items():
    print(f"The model with the minimal MSE for {index} is {model}, the MSE value is {min_mse_value[index]:.2f}")

The model with the minimal MSE for nasdaq is arima, the MSE value is 351821.42
The model with the minimal MSE for sp500 is gru, the MSE value is 9537.11
The model with the minimal MSE for dji is sru, the MSE value is 187336.65
